In [1]:
pip install pandas numpy yfinance scikit-learn requests

  Using cached numpy-2.2.2-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached scikit_learn-1.6.1-cp310-cp310-win_amd64.whl.metadata (15 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
     ---------------------------------------- 0.0/948.2 kB ? eta -:--:--
     ---------------------------------------- 0.0/948.2 kB ? eta -:--:--
     ----------- ---------------------------- 262.1/948.2 kB ? eta -:--:--
     ----------- ---------------------------- 262.1/948.2 kB ? eta -:--:--
     ------------------- ---------------- 524.3/948.2 kB 728.2 kB/s eta 0:00:01
     -------------------------------------- 948.2/948.2 kB 1.0 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished w

Klasifikasi 

In [4]:
# Instal library yang diperlukan

import pandas as pd
import numpy as np
import yfinance as yf
import requests
from sklearn.ensemble import RandomForestClassifier
from datetime import datetime, timedelta

# ================== MENDAPATKAN DATA REAL-TIME ==================
def get_live_price():
    """Mendapatkan harga XRP terkini dalam USD dan IDR dari CoinGecko"""
    url = "https://api.coingecko.com/api/v3/simple/price"
    params = {
        'ids': 'ripple',
        'vs_currencies': 'usd,idr',
        'include_market_cap': 'false',
        'include_24hr_vol': 'false',
        'include_24hr_change': 'false'
    }
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Memastikan respons HTTP valid
        data = response.json()
        return data['ripple']['usd'], data['ripple']['idr']
    except Exception as e:
        print(f"Error fetching live price: {e}")
        return None, None

# ================== MENGAMBIL DATA HISTORIS ==================
def get_historical_data(days=365):
    """Mengambil data 1 tahun terakhir dari Yahoo Finance (dalam USD)"""
    end_date = datetime.today().strftime('%Y-%m-%d')
    start_date = (datetime.today() - timedelta(days=days)).strftime('%Y-%m-%d')
    try:
        data = yf.download("XRP-USD", start=start_date, end=end_date)
        if data.empty:
            raise ValueError("No historical data available for the given period.")
        return data
    except Exception as e:
        print(f"Error fetching historical data: {e}")
        return None

# ================== MEMPROSES FITUR TEKNIKAL ==================
def calculate_technical_indicators(df):
    df['SMA_20'] = df['Close'].rolling(20).mean()
    df['SMA_50'] = df['Close'].rolling(50).mean()

    # Hitung RSI dengan Exponential Moving Average (EMA)
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.ewm(com=13, adjust=False).mean()
    avg_loss = loss.ewm(com=13, adjust=False).mean()

    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))

    # Hitung MACD
    ema12 = df['Close'].ewm(span=12, adjust=False).mean()
    ema26 = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = ema12 - ema26

    df.dropna(inplace=True)
    return df

# ================== MODEL & PREDIKSI ==================
def predict_tomorrow():
    # Dapatkan data
    df = get_historical_data()
    if df is None or df.empty:
        print("No historical data available. Prediction cannot be made.")
        return None

    df = calculate_technical_indicators(df)

    # Buat target
    df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)

    # Split data
    features = df[['SMA_20', 'SMA_50', 'RSI', 'MACD']]
    target = df['Target']

    # Pastikan ada cukup data untuk pelatihan
    if len(features) < 100:  # Misalnya, minimal 100 data
        print("Insufficient data for training the model.")
        return None

    X_train, X_test = features[:-1], features[-1:]
    y_train = target[:-1]

    # Latih model
    model = RandomForestClassifier(n_estimators=200, random_state=42)
    model.fit(X_train, y_train)

    # Evaluasi akurasi model
    accuracy = model.score(X_train, y_train)
    print(f"Model Accuracy: {accuracy * 100:.2f}%")

    # Prediksi
    prediction = model.predict(X_test)
    return prediction[0]

# ================== TAMPILKAN HASIL ==================
# ================== TAMPILKAN HASIL ==================
if __name__ == "__main__":
    # Dapatkan harga terkini
    current_price_usd, current_price_idr = get_live_price()
    if current_price_usd is None or current_price_idr is None:
        print("Failed to fetch live price.")
        exit()

    # Dapatkan prediksi
    prediction = predict_tomorrow()
    if prediction is None:
        print("Prediction failed.")
        exit()

    # Format angka Rupiah
    def format_idr(price):
        return "Rp {:,.0f}".format(price).replace(",", ".")

    # Definisikan warna ANSI
    GREEN = '\033[92m'  # Hijau
    RED = '\033[91m'    # Merah
    RESET = '\033[0m'   # Reset warna

    # Tampilkan hasil
    print("\n" + "="*55)
    print(f"🕒 Update Terkini: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"💰 Harga XRP (USD): ${current_price_usd:,.4f}")
    print(f"💰 Harga XRP (IDR): {format_idr(current_price_idr)}")
    print(f"🔮 Prediksi Besok: {GREEN}NAIK 📈{RESET}" if prediction == 1 else f"🔮 Prediksi Besok: {RED}TURUN 📉{RESET} (USD & IDR)")
    print("="*55 + "\n")

    # Catatan
    print("Catatan Penting:")
    print("- Prediksi dalam IDR mengikuti pergerakan harga USD")
    print("- Nilai tukar USD/IDR real-time bisa mempengaruhi hasil aktual")
    print("- Akurasi model sekitar 55-65% berdasarkan data historis")
    print("- Tidak disarankan untuk trading nyata tanpa analisis tambahan")

[*********************100%***********************]  1 of 1 completed


Model Accuracy: 100.00%

🕒 Update Terkini: 2025-02-04 12:21:12
💰 Harga XRP (USD): $2.5900
💰 Harga XRP (IDR): Rp 42.434
🔮 Prediksi Besok: TURUN 📉 (USD & IDR)

Catatan Penting:
- Prediksi dalam IDR mengikuti pergerakan harga USD
- Nilai tukar USD/IDR real-time bisa mempengaruhi hasil aktual
- Akurasi model sekitar 55-65% berdasarkan data historis
- Tidak disarankan untuk trading nyata tanpa analisis tambahan


Regresi

In [5]:
# Instal library yang diperlukan
# pip install pandas numpy yfinance scikit-learn requests

import pandas as pd
import numpy as np
import yfinance as yf
import requests
from sklearn.ensemble import RandomForestRegressor
from datetime import datetime, timedelta

# ================== MENDAPATKAN DATA REAL-TIME ==================
def get_live_price():
    """Mendapatkan harga XRP terkini dalam USD dan IDR dari CoinGecko"""
    url = "https://api.coingecko.com/api/v3/simple/price"
    params = {
        'ids': 'ripple',
        'vs_currencies': 'usd,idr',
        'include_market_cap': 'false',
        'include_24hr_vol': 'false',
        'include_24hr_change': 'false'
    }
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Memastikan respons HTTP valid
        data = response.json()
        return data['ripple']['usd'], data['ripple']['idr']
    except Exception as e:
        print(f"Error fetching live price: {e}")
        return None, None

# ================== MENGAMBIL DATA HISTORIS ==================
def get_historical_data(days=365):
    """Mengambil data 1 tahun terakhir dari Yahoo Finance (dalam USD)"""
    end_date = datetime.today().strftime('%Y-%m-%d')
    start_date = (datetime.today() - timedelta(days=days)).strftime('%Y-%m-%d')
    try:
        data = yf.download("XRP-USD", start=start_date, end=end_date)
        if data.empty:
            raise ValueError("No historical data available for the given period.")
        return data
    except Exception as e:
        print(f"Error fetching historical data: {e}")
        return None

# ================== MEMPROSES FITUR TEKNIKAL ==================
def calculate_technical_indicators(df):
    df['SMA_20'] = df['Close'].rolling(20).mean()
    df['SMA_50'] = df['Close'].rolling(50).mean()

    # Hitung RSI dengan Exponential Moving Average (EMA)
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.ewm(com=13, adjust=False).mean()
    avg_loss = loss.ewm(com=13, adjust=False).mean()

    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))

    # Hitung MACD
    ema12 = df['Close'].ewm(span=12, adjust=False).mean()
    ema26 = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = ema12 - ema26

    df.dropna(inplace=True)
    return df

# ================== MODEL & PREDIKSI ==================
def predict_tomorrow():
    # Dapatkan data
    df = get_historical_data()
    if df is None or df.empty:
        print("No historical data available. Prediction cannot be made.")
        return None, None, None

    df = calculate_technical_indicators(df)

    # Buat target untuk harga tertinggi dan terendah
    df['High_Target'] = df['High'].shift(-1)  # Harga tertinggi besok
    df['Low_Target'] = df['Low'].shift(-1)   # Harga terendah besok

    # Split data
    features = df[['SMA_20', 'SMA_50', 'RSI', 'MACD']]
    high_target = df['High_Target']
    low_target = df['Low_Target']

    # Pastikan ada cukup data untuk pelatihan
    if len(features) < 100:  # Misalnya, minimal 100 data
        print("Insufficient data for training the model.")
        return None, None, None

    X_train_high, X_test_high = features[:-1], features[-1:]
    y_train_high = high_target[:-1]

    X_train_low, X_test_low = features[:-1], features[-1:]
    y_train_low = low_target[:-1]

    # Latih model untuk harga tertinggi
    model_high = RandomForestRegressor(n_estimators=200, random_state=42)
    model_high.fit(X_train_high, y_train_high)

    # Latih model untuk harga terendah
    model_low = RandomForestRegressor(n_estimators=200, random_state=42)
    model_low.fit(X_train_low, y_train_low)

    # Prediksi harga tertinggi dan terendah
    predicted_high = model_high.predict(X_test_high)[0]
    predicted_low = model_low.predict(X_test_low)[0]

    return predicted_high, predicted_low

# ================== TAMPILKAN HASIL ==================
if __name__ == "__main__":
    # Dapatkan harga terkini
    current_price_usd, current_price_idr = get_live_price()
    if current_price_usd is None or current_price_idr is None:
        print("Failed to fetch live price.")
        exit()

    # Dapatkan prediksi harga tertinggi dan terendah
    predicted_high, predicted_low = predict_tomorrow()
    if predicted_high is None or predicted_low is None:
        print("Prediction failed.")
        exit()

    # Format angka Rupiah
    def format_idr(price):
        return "Rp {:,.0f}".format(price).replace(",", ".")

    # Definisikan warna ANSI
    GREEN = '\033[92m'  # Hijau
    RED = '\033[91m'    # Merah
    RESET = '\033[0m'   # Reset warna

    # Tampilkan hasil
    print("\n" + "="*70)
    print(f"🕒 Update Terkini: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"💰 Harga XRP (USD): ${current_price_usd:,.4f}")
    print(f"💰 Harga XRP (IDR): {format_idr(current_price_idr)}")
    print(f"📈 Prediksi Harga Tertinggi Besok: ${predicted_high:.4f} (USD)")
    print(f"📉 Prediksi Harga Terendah Besok: ${predicted_low:.4f} (USD)")
    print("="*70 + "\n")

    # Catatan
    print("Catatan Penting:")
    print("- Prediksi harga tertinggi dan terendah berdasarkan data historis.")
    print("- Akurasi model sekitar 60-70% tergantung pada volatilitas pasar.")
    print("- Tidak disarankan untuk trading nyata tanpa analisis tambahan.")

[*********************100%***********************]  1 of 1 completed



🕒 Update Terkini: 2025-02-04 13:13:56
💰 Harga XRP (USD): $2.5600
💰 Harga XRP (IDR): Rp 41.822
📈 Prediksi Harga Tertinggi Besok: $3.0032 (USD)
📉 Prediksi Harga Terendah Besok: $2.6263 (USD)

Catatan Penting:
- Prediksi harga tertinggi dan terendah berdasarkan data historis.
- Akurasi model sekitar 60-70% tergantung pada volatilitas pasar.
- Tidak disarankan untuk trading nyata tanpa analisis tambahan.


In [8]:
# Instal library yang diperlukan
# pip install pandas numpy yfinance scikit-learn requests

import pandas as pd
import numpy as np
import yfinance as yf
import requests
from sklearn.ensemble import RandomForestRegressor
from datetime import datetime, timedelta

# ================== MENDAPATKAN DATA REAL-TIME ==================
def get_live_price():
    """Mendapatkan harga XRP terkini dalam USD dan IDR dari CoinGecko"""
    url = "https://api.coingecko.com/api/v3/simple/price"
    params = {
        'ids': 'ripple',
        'vs_currencies': 'usd,idr',
        'include_market_cap': 'false',
        'include_24hr_vol': 'false',
        'include_24hr_change': 'false'
    }
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Memastikan respons HTTP valid
        data = response.json()
        return data['ripple']['usd'], data['ripple']['idr']
    except Exception as e:
        print(f"Error fetching live price: {e}")
        return None, None

# ================== MENGAMBIL DATA INTRADAY ==================
def get_intraday_data(days=7, interval="60m"):
    """Mengambil data intraday dari Yahoo Finance (per jam)"""
    end_date = datetime.today()
    start_date = end_date - timedelta(days=days)
    try:
        data = yf.download("XRP-USD", start=start_date, end=end_date, interval=interval)
        if data.empty:
            raise ValueError("No intraday data available for the given period.")
        return data
    except Exception as e:
        print(f"Error fetching intraday data: {e}")
        return None

# ================== MEMPROSES FITUR TEKNIKAL ==================
def calculate_technical_indicators(df):
    df['SMA_20'] = df['Close'].rolling(20).mean()
    df['SMA_50'] = df['Close'].rolling(50).mean()

    # Hitung RSI dengan Exponential Moving Average (EMA)
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.ewm(com=13, adjust=False).mean()
    avg_loss = loss.ewm(com=13, adjust=False).mean()

    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))

    # Hitung MACD
    ema12 = df['Close'].ewm(span=12, adjust=False).mean()
    ema26 = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = ema12 - ema26

    df.dropna(inplace=True)
    return df

# ================== MODEL & PREDIKSI ==================
def predict_tomorrow():
    # Dapatkan data intraday
    df = get_intraday_data(days=7, interval="60m")
    if df is None or df.empty:
        print("No intraday data available. Prediction cannot be made.")
        return None, None, None, None, None

    df = calculate_technical_indicators(df)

    # Buat target untuk harga tertinggi, terendah, dan waktunya
    df['High_Target'] = df['High'].shift(-1)  # Harga tertinggi besok
    df['Low_Target'] = df['Low'].shift(-1)   # Harga terendah besok
    df['Time'] = df.index.time               # Waktu (jam:menit:detik)

    # Split data
    features = df[['SMA_20', 'SMA_50', 'RSI', 'MACD']]
    high_target = df['High_Target']
    low_target = df['Low_Target']

    # Pastikan ada cukup data untuk pelatihan
    if len(features) < 100:  # Misalnya, minimal 100 data
        print("Insufficient data for training the model.")
        return None, None, None, None, None

    X_train_high, X_test_high = features[:-1], features[-1:]
    y_train_high = high_target[:-1]

    X_train_low, X_test_low = features[:-1], features[-1:]
    y_train_low = low_target[:-1]

    # Latih model untuk harga tertinggi
    model_high = RandomForestRegressor(n_estimators=200, random_state=42)
    model_high.fit(X_train_high, y_train_high)

    # Latih model untuk harga terendah
    model_low = RandomForestRegressor(n_estimators=200, random_state=42)
    model_low.fit(X_train_low, y_train_low)

    # Prediksi harga tertinggi dan terendah
    predicted_high = model_high.predict(X_test_high)[0]
    predicted_low = model_low.predict(X_test_low)[0]

    # Prediksi waktu tertinggi dan terendah (berdasarkan pola historis)
    time_high = df['Time'].iloc[df['High'].idxmax()]
    time_low = df['Time'].iloc[df['Low'].idxmin()]

    return predicted_high, predicted_low, time_high, time_low

# ================== TAMPILKAN HASIL ==================
if __name__ == "__main__":
    # Dapatkan harga terkini
    current_price_usd, current_price_idr = get_live_price()
    if current_price_usd is None or current_price_idr is None:
        print("Failed to fetch live price.")
        exit()

    # Dapatkan prediksi harga tertinggi, terendah, dan waktunya
    predicted_high, predicted_low, time_high, time_low = predict_tomorrow()
    if predicted_high is None or predicted_low is None:
        print("Prediction failed.")
        exit()

    # Format angka Rupiah
    def format_idr(price):
        return "Rp {:,.0f}".format(price).replace(",", ".")

    # Definisikan warna ANSI
    GREEN = '\033[92m'  # Hijau
    RED = '\033[91m'    # Merah
    RESET = '\033[0m'   # Reset warna

    # Tampilkan hasil
    print("\n" + "="*80)
    print(f"🕒 Update Terkini: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"💰 Harga XRP (USD): ${current_price_usd:,.4f}")
    print(f"💰 Harga XRP (IDR): {format_idr(current_price_idr)}")
    print(f"📈 Prediksi Harga Tertinggi Besok: ${predicted_high:.4f} (USD) pada jam {time_high}")
    print(f"📉 Prediksi Harga Terendah Besok: ${predicted_low:.4f} (USD) pada jam {time_low}")
    print("="*80 + "\n")

    # Catatan
    print("Catatan Penting:")
    print("- Prediksi harga tertinggi dan terendah berdasarkan data historis.")
    print("- Akurasi model sekitar 60-70% tergantung pada volatilitas pasar.")
    print("- Tidak disarankan untuk trading nyata tanpa analisis tambahan.")

[*********************100%***********************]  1 of 1 completed

Error fetching live price from Binance: HTTPSConnectionPool(host='api.binance.com', port=443): Max retries exceeded with url: /api/v3/ticker/price?symbol=XRPUSDT (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000244FC7E1990>, 'Connection to api.binance.com timed out. (connect timeout=None)'))


Error fetching historical data from Binance: HTTPSConnectionPool(host='api.binance.com', port=443): Max retries exceeded with url: /api/v3/klines?symbol=XRPUSDT&interval=1h&startTime=1738047182906&endTime=1738651982906&limit=1000 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000244FC926830>, 'Connection to api.binance.com timed out. (connect timeout=None)'))
Failed to fetch all required data sources.

🕒 Update Terkini: 2025-02-04 13:53:24
💰 Harga XRP (USD - CoinGecko): $2.5800
💰 Harga XRP (IDR - CoinGecko): Rp 42.248


TypeError: unsupported format string passed to NoneType.__format__

: 